# 加载模型以及数据

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOTrainer, GRPOConfig  # 假设 trl 库中有 GRPOTrainer 模块
from peft import PeftModel
from reward import compute_rewards
def load_data(input_path):
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split('<think>', 1)
            if len(parts) != 2:
                print(f"警告: 格式错误的行，已跳过: {line}")
                continue
            keywords_part, lyrics = parts[0], parts[1]
            keywords = [kw.strip() for kw in keywords_part.split('，')]

            # 关键修改：使用关键词作为 prompt，歌词作为 completion
            data.append({
                'prompt': "根据以下关键词生成一首歌词，歌词中包含多个句子，确保句子通顺,诗意,格式正确.让我们一步一步的思考（思考过程包含在<think>和</think>之间）：" + ",".join(keywords),  # 关键词拼接成字符串，作为模型输入
                'completion':  "<think>" + lyrics,  # 歌词（去掉多余空格），作为模型输出
                'keywords': keywords,  # 关键词拼接成字符串，作为模型输入
            })
    
    print(f"成功加载 {len(data)} 条数据")
    return data




E:\共享\GoodMusicV2.0\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_data('../data/CoTdata.txt')
if dataset:
    print("第一条数据:", dataset[0])
else:
    print("未加载到有效数据。")

成功加载 1000 条数据
第一条数据: {'prompt': '根据以下关键词生成一首歌词，歌词中包含多个句子，确保句子通顺,诗意,格式正确.让我们一步一步的思考（思考过程包含在<think>和</think>之间）：午夜,寒冬,心动', 'completion': '<think>嗯，用户让我根据“午夜，寒冬，心动”这三个关键词写一首诗。首先，我需要理解每个词带来的意象和情感。午夜通常给人一种寂静、神秘的感觉，可能带有孤独或反思的情绪。寒冬则联想到寒冷、冰雪，可能还有萧瑟的景象，但也可以有温暖的反差，比如炉火或者温暖的回忆。心动是情感的核心，可能涉及爱情、突然的情感波动，或者内心的温暖。接下来，我要考虑如何将这三个元素融合在一起。可能需要营造一个寒冷的冬夜环境，然后在这样的背景下引入心动的瞬间，形成对比。比如，用寒冷的景象来衬托内心的温暖或突然的情感变化。然后，结构方面，可能需要分几段，每段围绕一个关键词展开，但又要相互联系。比如，前两段描绘午夜和寒冬的景象，第三段引入心动的元素。或者交替出现，交织在一起。语言风格方面，用户可能希望有诗意和意象化的表达，避免直白。使用比喻、拟人等修辞手法，比如将心跳比作钟摆，或者用冰晶、霜花来表现寒冷。同时，注意押韵和节奏，使诗歌流畅。还需要考虑用户可能的深层需求。他们可能不仅仅想要一首描述这三个词的诗，而是希望表达在寒冷孤独的环境中突然感受到的温暖或爱情，强调对比和情感冲击。因此，诗里需要有转折点，从冷到暖，从静到动。可能会遇到的挑战是如何自然地将三个关键词串联起来，不显得生硬。可能需要找到一个共同的场景或隐喻，比如午夜寒冬中的相遇，或者内心的变化。同时，避免陈词滥调，用新颖的比喻，比如将霜花比作时针，或者心跳与钟摆的共振。最后，检查诗的整体意境是否统一，情感是否连贯，是否有足够的意象让读者产生共鸣。可能需要调整用词和结构，使每个部分都服务于主题，增强整体效果。</think>:《冰裂时刻》/石英血管里霜花正逆向生长/成为时针，成为所有表盘/反向旋转的证词。玻璃在窗框深处/爆出细纹，像候鸟迁徙时/骨骼深处迸发的裂响//壁炉吞下最后一块松木的年轮/灰烬在铁架上凝结成第六种晶体/这时钟摆突然停驻——/某种振频正穿过雪原/穿过二十年冻土下沉默的矿脉//冰层深处传来远古鲸群的应和/我们站在地磁偏转的切线上/听见彼此胸腔里/石英开始顺时

In [3]:
base_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B").to("cuda")
# 2. 加载 LoRA 适配器
model = PeftModel.from_pretrained(base_model, "../3_26_LoRA").to("cuda")  # 你的 LoRA 路径
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
tokenizer.pad_token = tokenizer.eos_token
# Load LoRA   lora_config = LoraConfig(       task_type="CAUSAL_LM",       r=16,       lora_alpha=32,       target_modules="all-linear",   )   model = get_peft_model(model, lora_config)   print(model.print_trainable_parameters())   
from peft import LoraConfig, get_peft_model
target_modules = ["q_proj", "k_proj", "v_proj"] 
lora_config = LoraConfig(
    r=2,                   # 秩（可尝试8~32）
    lora_alpha=32,          # 缩放系数（通常设为2*r）
    target_modules=target_modules,  
    bias="none",            # 不训练偏置项
)
model = get_peft_model(model, lora_config)   
print(model.print_trainable_parameters()) 

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


trainable params: 372,736 || all params: 1,777,460,736 || trainable%: 0.0210
None


E:\共享\GoodMusicV2.0\.venv\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
E:\共享\GoodMusicV2.0\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


# 配置训练参数

In [4]:
# 配置 GRPO 的超参数（这里的参数可以根据需要进行调整）
config = GRPOConfig(
    gradient_accumulation_steps = 50,    # 多少步更新一次参考模型
    per_device_train_batch_size=2,           # 每个批次的样本数
    epsilon=0.2,             # GRPO 中的 clip 范围
    beta=0.05,               # KL 惩罚系数
    num_train_epochs=1,     # 总训练步数(总周期)
    num_generations=2,           # 分组采样的大小
    learning_rate=1e-5,       # 优化器的学习率
    bf16=True,     
    adam_beta1=0.9,
    adam_beta2=0.98,
    optim="adamw_8bit", # 优化器
    max_grad_norm=0.1,        # 梯度裁剪的最大值
    save_steps=1000,           # 多少步保存一次模型
    save_total_limit=2,       # 最多保存几个模型         
    logging_steps=5,         # 多少步打印一次训练信息
    output_dir="GRPO",             # 模型保存路径
    weight_decay=0.01,     # 权重衰减
    warmup_ratio=0.03,       # 预热比例
    max_prompt_length=256,
    max_completion_length=1024, # 最大输出长度
    report_to='tensorboard', # or `tensorboard`
)
# Training arguments   training_args = GRPOConfig(       
# output_dir="GRPO",    
# learning_rate=2e-5,   
# per_device_train_batch_size=8, 
# gradient_accumulation_steps=2,     
# max_prompt_length=512,    
# max_completion_length=96,     
# num_generations=8,     
# optim="adamw_8bit",      
# num_train_epochs=1,      
# bf16=True,     
# report_to=["wandb"],   
# remove_unused_columns=False,    
# logging_steps=1,   
# )   

# 训练模型

In [5]:
# Trainer   trainer = GRPOTrainer(    
# model=model,   
# reward_funcs=[reward_len], 
# args=training_args,    
# train_dataset=dataset["train"],  
# )      
# Train model  
# wandb.init(project="GRPO") 
# trainer.train()
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[compute_rewards],
    args=config,
    train_dataset=dataset
)
trainer.train()

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


KeyboardInterrupt: 

# 评估

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

def plot_training_metrics(losses, kls, avg_rewards, output_dir="."):
    """
    绘制并保存训练指标图表
    
    参数:
        losses: 训练损失列表
        kls: KL散度列表
        avg_rewards: 平均奖励列表
        output_dir: 输出目录路径
    """
    # 生成带时间戳的唯一文件名
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"{output_dir}/training_curves_{timestamp}.png"
    
    # 创建画布
    plt.figure(figsize=(15, 5), dpi=300)
    
    # 1. Loss 曲线
    plt.subplot(1, 3, 1)
    plt.plot(losses, label="Loss", linewidth=1.5, color='blue')
    plt.title("Training Loss", fontsize=10)
    plt.xlabel("Step", fontsize=9)
    plt.ylabel("Loss", fontsize=9)
    plt.grid(True, alpha=0.3)
    
    # 2. KL 散度曲线
    plt.subplot(1, 3, 2)
    plt.plot(kls, label="KL Divergence", linewidth=1.5, color='orange')
    plt.title("KL Divergence", fontsize=10)
    plt.xlabel("Step", fontsize=9)
    plt.ylabel("KL Divergence", fontsize=9)
    plt.grid(True, alpha=0.3)
    
    # 3. 平均奖励曲线
    plt.subplot(1, 3, 3)
    plt.plot(avg_rewards, label="Avg Reward", linewidth=1.5, color='green')
    plt.title("Average Reward", fontsize=10)
    plt.xlabel("Step", fontsize=9)
    plt.ylabel("Reward", fontsize=9)
    plt.grid(True, alpha=0.3)
    
    # 调整布局并保存
    plt.tight_layout()
    plt.savefig(
        output_path,
        bbox_inches='tight',
        facecolor='white',
        dpi=300
    )
    plt.close()
    
    print(f"训练指标图表已保存至: {output_path}")

# 使用示例 (假设你已经有了这些数据)
# losses = [...]  # 你的损失数据
# kls = [...]     # 你的KL散度数据
# avg_rewards = [...]  # 你的平均奖励数据
# plot_training_metrics(losses, kls, avg_rewards)



In [ ]:
class MetricsCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.metrics = {
            'loss': [], 
            'kl_divergence': [], 
            'avg_reward': []
        }
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if 'loss' in logs:
                self.metrics['loss'].append(logs['loss'])
            if 'kl_divergence' in logs:
                self.metrics['kl_divergence'].append(logs['kl_divergence'])
            if 'rewards' in logs:  # 假设返回的是列表，取其平均值
                avg_reward = sum(logs['rewards'])/len(logs['rewards'])
                self.metrics['avg_reward'].append(avg_reward)
                
                

In [ ]:
plot_training_metrics(metrics_callback.metrics['loss'],metrics_callback.metrics['kl_divergence'],metrics_callback.metrics['avg_reward'])